### Parallel Workflow with Microsoft Agent Framework and Azure AI Foundry

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4 agent-framework[viz]

### Setting Up the Environment 

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from agent_framework import Executor, WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, handler
from agent_framework import WorkflowBuilder, WorkflowViz

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Defining a Function to Create a Chat Agent

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_agent(agent_name: str,
                       agent_instructions: str) -> ChatAgent:
    
    # Create async Azure credential
    credential = AzureCliCredential()
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIAgentClient(async_credential=credential,
                                     project_endpoint=project_endpoint,
                                     model_deployment_name=model)

    try:

        # Create the Multi-Tool Agent
        agent = chat_client.create_agent(
            name=agent_name,
            instructions=agent_instructions,
        )

        print("{} Agent created successfully!".format(agent_name))
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()


### Creating the Location Picker Agent

In [ ]:
location_picker_agent = await create_agent(
    agent_name = "Location-Picker-Agent",
    agent_instructions = """You are a helpful assistant that helps users pick a location for their vacation."""
)

### Creating the Destination Recommender Agent

In [ ]:
destination_recommender_agent = await create_agent(
    agent_name = "Destination-Recommender-Agent",
    agent_instructions = """You are a travel expert that provides personalized vacation recommendations based on user preferences and locations. """
)

### Creating the Weather Agent

In [ ]:
weather_agent = await create_agent(
    agent_name = "Weather-Agent",
    agent_instructions = """You are a weather expert that provides accurate and up-to-date weather information for various locations selected"""
)

### Creating the Cuisine Suggestion Agent

In [ ]:
cuisine_suggestion_agent = await create_agent(
    agent_name = "Cuisine-Suggestion-Agent",
    agent_instructions = """You are a culinary expert that suggests popular local cuisines and dining options based on the selected vacation destinations."""
)

### Creating the Itinerary Planner Agent

In [ ]:
itinerary_planner_agent = await create_agent(
    agent_name = "Itinerary-Planner-Agent",
    agent_instructions = """You are an itinerary planning expert that creates detailed travel itineraries based on user preferences, selected destinations, weather conditions, and local cuisine options."""
)

### Creating the Location Selector Agent Executor

In [ ]:
class LocationSelectorExecutor(Executor):

    @handler
    async def handle(self, user_query: str, ctx: WorkflowContext[str]) -> None:
        response = await location_picker_agent.run(user_query)

        await ctx.send_message(str(response))

### Creating the Destination Recommender Agent Executor

In [ ]:
class DestinationRecommenderExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await destination_recommender_agent.run(location)

        await ctx.send_message(str(response))

### Creating the Weather Agent Executor

In [ ]:
class WeatherExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await weather_agent.run(location)

        await ctx.send_message(str(response))

### Creating the Cuisine Suggestion Agent Executor

In [ ]:
class CuisineSuggestionExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await cuisine_suggestion_agent.run(location)

        await ctx.send_message(str(response))

### Creating the Itinerary Planner Agent Executor

In [ ]:
class ItineraryPlannerExecutor(Executor):

    @handler
    async def handle(self, results: list[str], ctx: WorkflowContext[str]) -> None:
        response = await itinerary_planner_agent.run(results)

        await ctx.yield_output(str(response))

### Building the Parallel Workflow

In [ ]:
# Create the executor instances / objects
location_selector_executor = LocationSelectorExecutor(id="LocationSelector")
destination_recommender_executor = DestinationRecommenderExecutor(id="DestinationRecommender")
weather_executor = WeatherExecutor(id="Weather")
cuisine_suggestion_executor = CuisineSuggestionExecutor(id="CuisineSuggestion")
itinerary_planner_executor = ItineraryPlannerExecutor(id="ItineraryPlanner")

# Build the workflow
workflow = (
    WorkflowBuilder()
    .set_start_executor(location_selector_executor)
    .add_fan_out_edges(location_selector_executor, [destination_recommender_executor, weather_executor, cuisine_suggestion_executor])
    .add_fan_in_edges([destination_recommender_executor, weather_executor, cuisine_suggestion_executor], itinerary_planner_executor)
    .build()
)

viz = WorkflowViz(workflow)

### Generating Mermaid Diagram for Visualization

In [ ]:
mermaid_content = viz.to_mermaid()

# printing mermaid content as markdown
from IPython.display import Markdown, display
display(Markdown(f"```mermaid\n{mermaid_content}\n```"))

### Running the Workflow and Streaming Events

In [ ]:
# Run the workflow and stream events in notebook
async def main():
    async for event in workflow.run_stream("help me plan a vacation to India with the following details: I love historical sites, prefer warm weather, and enjoy trying local foods."):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")

await main()
